In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
print(os.getpid())
%cd ../../

import pylib.mix as mix
import pylib.measurement as mse
reload = mix.reload_module

path_qucf_ = "../QuCF/tests/test_phase_estimation/"

In [ ]:
# -----------------------------------------------
# --- Helper functions ---
# -----------------------------------------------
def plot_probabilities_of_meas_phases(om, eigphase_ref):
    probs_meas = om.probs_
    N_y_points = 1 << int(om.constants_["ny"])
    int_ys = np.array(range(N_y_points))

    qucf_eig_phases = 2.*np.pi * int_ys / N_y_points

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(
        qucf_eig_phases, 
        probs_meas, 
        color='b', marker="o", linestyle = ':', label = "outcomes"
    )
    ax.axvline(x=eigphase_ref, linestyle = ':', color='r', linewidth = 2, label="sought eigenphase")
    plt.xlabel("outcome phases")
    plt.title("probability distribution of measurement outcomes")
    ax.legend()
    plt.grid(True)
    plt.show()
    return
# -----------------------------------------------
def get_ref_eigenphase_case_1(om):
    th = om.constants_["th"]

    A = mix.Ry(th)
    eig_vals, eig_vecs = np.linalg.eig(A)

    print("eigenvectors")
    mix.print_matrix(1j * np.transpose(eig_vecs))
    print()

    eig_phases = np.angle(eig_vals)
    print("eigphases: ", eig_phases)
    eigphase_ref = eig_phases[0]

    return eigphase_ref
# -----------------------------------------------
def get_ref_eigenphase_case_3(om):
    th1 = om.constants_["th1_"]
    th2 = om.constants_["th2_"]
    th3 = om.constants_["th3_"]

    A = np.kron(mix.Rz(th3), np.kron(mix.Rz(th2), mix.Rz(th1)))
    eig_vals, eig_vecs = np.linalg.eig(A)

    print("eigenvectors: ")
    mix.print_matrix(np.transpose(eig_vecs))
    print()

    eig_phases = np.angle(eig_vals)
    print("eigphases: ")
    mix.print_array(eig_phases, n_in_row = 8)

    eigphase_ref = 2.*np.pi + eig_phases[-1]
    return eigphase_ref
# -----------------------------------------------

In [ ]:
# ----------------------------------------------------------------------
# --- 1-QUBIT GATE: Read results from the C++ sim. of the PE circuit ---
# ----------------------------------------------------------------------
reload(mse)

om1 = mse.MeasOracle__(True)
om1.path_ = path_qucf_
om1.pname_ = "PE1"
om1.open()

eigphase_ref1_ = get_ref_eigenphase_case_1(om1)
print()
plot_probabilities_of_meas_phases(om1, eigphase_ref1_)

In [ ]:
# ----------------------------------------------------------------------
# --- 3-QUBIT GATE: Read results from the C++ sim. of the PE circuit ---
# ----------------------------------------------------------------------
reload(mse)

om3 = mse.MeasOracle__()
om3.path_ = path_qucf_
om3.pname_ = "PE3"
om3.open()

# eigphase_ref3_ = om3.constants_["sought_eigphase"]
eigphase_ref3_ = get_ref_eigenphase_case_3(om3)
print()
plot_probabilities_of_meas_phases(om3, eigphase_ref3_)